In [1]:
import os
import pandas as pd
from pandas import HDFStore
from nltk.corpus import stopwords
import IPython

In [2]:
from common.functions import get_highlighted_function_code
from common.settings import embedding_dim, seq_length, data_access
# from common.classes import VqaSpecs
from common.utils import VerboseTimer
import vqa_logger 
from common.os_utils import File
from pre_processing.meta_data import create_meta
pd.set_option('display.max_colwidth', -1)

### Preprocessing and creating meta data

Get the data itself, Note the only things required in dataframe are:
1. image_name
2. processed question
3. processed answer


In [3]:
# index	image_name	question	answer	group	path	original_question	original_answer	tumor	hematoma	brain	abdomen	neck	liver	imaging_device	answer_embedding	question_embedding	is_imaging_device_question
df_data = data_access.load_processed_data(columns=['path','question','answer', 'processed_question','processed_answer', 'group','question_category'])        
df_data = df_data[df_data.group.isin(['train','validation', 'test'])]
print(f'Data length: {len(df_data)}')        

[2019-04-22 14:09:59][data_access.api][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-04-22 14:09:59][data_access.api][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-04-22 14:09:59][common.utils][DEBUG] Starting 'Loading parquet'
[2019-04-22 14:09:59][common.utils][DEBUG] Loading parquet: 0:00:00.012558
[2019-04-22 14:09:59][common.utils][DEBUG] Starting 'Converting to pandas'
[2019-04-22 14:09:59][common.utils][DEBUG] Converting to pandas: 0:00:00.012302
Data length: 15292


In [4]:
df_data.sample(7)

,path,question,answer,processed_question,processed_answer,question_category,group
504,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic20934.jpg,what type of image modality is this?,xr - plain film,what type of image modality is this,xr plain film,Modality,train
1804,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic49854.jpg,what is the mr weighting in this image?,t1,what is the mr weighting in this image,t1,Modality,train
3633,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic54541.jpg,what type of imaging modality is shown?,nm - nuclear medicine,what type of imaging modality is shown,nm nuclear medicine,Modality,train
3980,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic9634.jpg,in what plane is this gastrointestinal image captured?,frontal,in what plane is this gastrointestinal image captured,frontal,Plane,train
7498,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic49404.jpg,which organ system is imaged?,skull and contents,which organ system is imaged,skull and contents,Organ,train
7886,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic53956.jpg,what is the organ system in this image?,skull and contents,what is the organ system in this image,skull and contents,Organ,train
8780,C:\Users\Public\Documents\Data\2019\train\Train_images\synpic21760.jpg,the x-ray shows what organ system?,musculoskeletal,the ray shows what organ system,musculoskeletal,Organ,train


#### We will use this function for creating meta data:

In [5]:
code = get_highlighted_function_code(create_meta,remove_comments=False)
IPython.display.display(code)  

In [6]:
print("----- Creating meta -----")
meta_data_dict = create_meta(df_data)

----- Creating meta -----
[2019-04-22 14:09:59][pre_processing.meta_data][DEBUG] Data frame had 14792 rows


#### Saving the data, so later on we don't need to compute it again

In [7]:
print("----- Saving meta -----")
data_access.save_meta(meta_data_dict)

----- Saving meta -----
[2019-04-22 14:10:00][data_access.api][DEBUG] Meta number of unique answers: 1675
[2019-04-22 14:10:00][data_access.api][DEBUG] Meta number of unique words: 2073


##### Test Loading:

In [43]:
loaded_meta = data_access.load_meta()
answers_meta = loaded_meta['answers']
words_meta = loaded_meta['words']


answers_meta.question_category.describe()
# words_meta.question_category.describe()

# answers_meta.sample(5)
# words_meta.sample(5)

# words_meta.question_category.drop_duplicates()

count     1675       
unique    5          
top       Abnormality
freq      1604       
Name: question_category, dtype: object